<h3>Tensor Flow를 활용한 데이터 회귀 분석</h3>

텐서플로우(TensorFlow)를 사용하여 기후 데이터를 기반으로 뉴스 수를 예측하는 코드

In [1]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import LogCosh
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import sys

In [2]:
# 출력 파일 설정
output_file = f"model_result.txt"
best_result = [float('inf'), float('inf'), None, None]

# 파일 열기
f = open(output_file, "w", encoding="utf-8")

for x in range(1,2):
    # 데이터 로드
    # 뉴스 데이터 로드
    datasPath = os.path.join(os.getcwd(), "..", "data_preprocessing", f"merged_final_data_{x*7}_shifted.csv")
    print(f"file path : {datasPath}")
    data = pd.read_csv(datasPath, encoding='utf-8')

    # Week_Num 속성 제거
    data = data.drop(columns=['Week_Num'])

    data.head()

    # 입력과 출력 컬럼 정의
    X_columns = ['datetime_x', 'datetime_y', 'Article_Num', 'Polution', 'Enviroment_Polution', 'Biodiversity_Loss',
                'Acid_Rain', 'Water_Pollution', 'Climate_Crisis', 'Accelerated_Global_Warming',
                'Ozone_Layer_Depletion', 'Hazardous_Substance_Leakage', 'Carbon_Dioxide',
                'Weekly_News_Count', 'News_Ratio']
    y_columns = ['tempmax', 'tempmin', 'temp', 'dew', 'humidity', 'precip', 
                'windspeed', 'sealevelpressure', 'moonphase']


    # String data인 datetime을 Unix Timestamp로 변환
    data['datetime_x'] = pd.to_datetime(data['datetime_x']).map(pd.Timestamp.timestamp)
    data['datetime_y'] = pd.to_datetime(data['datetime_y']).map(pd.Timestamp.timestamp)

    # 데이터 분리
    X = data[X_columns]
    y = data[y_columns]

    # 훈련 및 테스트 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    # 스케일링
    x_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()

    X_train = x_scaler.fit_transform(X_train)
    y_train = y_scaler.fit_transform(y_train)
    
    X_test = x_scaler.transform(X_test)
    y_test = y_scaler.transform(y_test)

    # datetime_x와 datetime_y를 타임스텝으로 처리하기 위해 reshape
    X_train = X_train.reshape(X_train.shape[0], 2, -1)  # (samples, timesteps=2, features/2)
    X_test = X_test.reshape(X_test.shape[0], 2, -1)     # (samples, timesteps=2, features/2)

    # 학습 과정 정의
    epoch = 1000
    lr = 0.0001

    # 사용자 정의 옵티마이저와 손실 함수
    # 손실 함수는 이상치에 대해 Log-Cosh Loss함수 사용
    # MAE와 유사하지만, 큰 오차에 대해 더 부드럽게 처리 가능
    optimizer = Adam(learning_rate=lr)  # Adam 옵티마이저, 학습률 0.001
    loss_function = LogCosh()

    # # 모델 생성
    # model = tf.keras.Sequential([
    #     tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    #     Dropout(0.15),
    #     tf.keras.layers.Dense(128, activation='relu'),
    #     Dropout(0.15),
    #     tf.keras.layers.Dense(64, activation='relu'),
    #     Dropout(0.15),
    #     tf.keras.layers.Dense(32, activation='relu'),
    #     Dropout(0.15),
    #     tf.keras.layers.Dense(len(y_columns))  # 출력 노드 수는 y_columns의 수와 동일
    # ])

    # 모델 생성
    model = Sequential([
        LSTM(256, activation='tanh', return_sequences=True, input_shape=(2, X_train.shape[2])),
        Dropout(0.12),
        LSTM(128, activation='tanh', return_sequences=False),
        Dropout(0.12),
        Dense(128, activation='relu'),
        Dropout(0.12),
        Dense(64, activation='relu'),
        Dropout(0.12),
        Dense(32, activation='relu'),
        Dropout(0.12),
        Dense(len(y_columns))  # 출력 노드 수는 y_columns의 수와 동일
    ])

    # EarlyStopping 콜백 정의
    early_stopping = EarlyStopping(
        monitor='val_loss',  # 검증 손실을 모니터링
        patience=10,         # 개선되지 않는 에포크 수 (침체기 기준)
        restore_best_weights=True,  # 최적 가중치를 복원
        verbose=1            # 종료 시 메시지 출력
    )
    
    # 모델 컴파일
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['mae'])

    # 모델 학습
    history = model.fit(X_train, y_train, epochs=epoch, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    try:
        # 모델 평가 
        test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
        print(f"==============={x*7} days matching===============")
        print(f"Test Loss: {test_loss:.4f}, Test MAE: {test_mae:.4f}")
        f.write(f"==============={x*7} days matching===============\n")
        f.write(f"Test Loss: {test_loss:.4f}, Test MAE: {test_mae:.4f}\n")

        # Best Model 저장
        if (best_result[0] > test_loss and best_result[1] > test_mae):
            best_result[0] = test_loss 
            best_result[1] = test_mae
            best_result[2] = x*7
            best_result[3] = model

        # 예측 수행
        y_pred = model.predict(X_test)

        # 출력 데이터(y_columns)에 대한 스케일러를 사용해 역변환
        y_test_inverse = y_scaler.inverse_transform(y_test)  # 실제 값 복원
        y_pred_inverse = y_scaler.inverse_transform(y_pred)  # 예측 값 복원

        # 예측 결과 출력
        print("Sample Predictions:")
        f.write("Sample Predictions:\n")
        for i in range(5):  # 처음 5개의 샘플에 대해 출력
            true_values = y_test_inverse[i]  # Numpy 배열에서 직접 값 가져오기
            predicted_values = y_pred_inverse[i]  # 예측 값
            print(f"Sample {i+1}:")
            f.write(f"Sample {i+1}:\n")
            for attr, true_val, pred_val in zip(y_columns, true_values, predicted_values):
                print(f"  {attr:<18} | True: {true_val} | Predicted: {pred_val}")
                f.write(f"  {attr:<18} | True: {true_val} | Predicted: {pred_val}\n")
            print()
            f.write("\n") 
    finally:
        print("FI")

print(f"best model is {best_result[2]}")
print(f"Test Loss: {best_result[0]:.4f}, Test MAE: {best_result[1]:.4f}")
f.write(f"best model is {best_result[2]}\n")
f.write(f"Test Loss: {best_result[0]:.4f}, Test MAE: {best_result[1]:.4f}\n")
# 저장할 모델이 best_result[3]에 있음
best_model = best_result[3]

# 저장 경로 지정
model_save_path = "best_model.h5"  # 파일 이름 및 경로 설정

# 모델 저장
best_model.save(model_save_path)

print(f"베스트 모델이 {model_save_path}에 저장되었습니다.")
f.close()

file path : C:\Users\limul\Desktop\College\2_grade\2_semester\computational_statistics\teamProject\2024-2-CSP\AI\made_by_LJW\..\data_preprocessing\merged_final_data_7_shifted.csv


ValueError: cannot reshape array of size 17055 into shape (1137,2,newaxis)